In [3]:
from darts.models import TCNModel
import pandas as pd
from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.utils.missing_values import fill_missing_values
from darts.utils.likelihood_models import GaussianLikelihood, QuantileRegression
# Assuming 'series' is your TimeSeries object
from darts import concatenate
from darts.utils.callbacks import TFMProgressBar
import numpy as np
import torch
from pytorch_lightning.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from darts.models import NBEATSModel
# from utils import fix_pythonpath_if_working_locally
from nixtla import NixtlaClient
from darts.models import NaiveDrift
from darts.models import StatsForecastAutoARIMA
def generate_torch_kwargs():
    # run torch models on CPU, and disable progress bars for all model stages except training.
    return {
        "pl_trainer_kwargs": {
            "accelerator": "cpu",
            "callbacks": [TFMProgressBar(enable_train_bar_only=True)],
        }
}
import os

In [4]:
data_dir = "C:/Users/mills/Documents/peru_dengue/province/python/data/"
output_dir = "C:/Users/mills/Documents/peru_dengue/province/python/output/"

In [5]:
quantiles = [0.010, 0.025, 0.050, 0.100, 0.150, 0.200, 0.250, 0.300, 0.350, 0.400, 0.450, 0.500, 0.550,  0.600, 0.650, 0.700, 0.750, 0.800, 0.850, 0.900, 0.950, 0.975, 0.990]

In [6]:
timeseries_df = pd.read_csv("C:/Users/mills/Documents/peru_dengue/province/python/data/ptl_province_inla_df.csv")
timeseries_df = timeseries_df[2:timeseries_df.shape[0]] #Remove first month where LAG values = 0
timeseries_df = timeseries_df.fillna(0)
province_names = timeseries_df['PROVINCE'].unique()

In [7]:
#Set up columns for covariates and 
covariate_cols = ['LAG_1_LOG_CASES', 
                  'LAG_1_tmin_roll_2', 'LAG_1_prec_roll_2'
                 ]
case_cols = ['LOG_CASES']

In [8]:
#If needed for working with standardised/scaled variables
scaler_case, scaler_covars = Scaler(), Scaler()

## 1) TCN MODEL Setup

In [9]:
tcn_model_name = "TCN_Model"
tcn_model = TCNModel(input_chunk_length=48,  # Use 48 months of history to predict the next month
                 output_chunk_length=1,  # Forecast 1 step ahead
                 kernel_size=3, 
                 num_filters=4, 
                 random_state=42, 
                   model_name=tcn_model_name,
                  likelihood = QuantileRegression(),
                save_checkpoints=True,
                     force_reset = True,
                 n_epochs=150, 
                 optimizer_kwargs={"lr": 1e-2})

## 2) TimeGPT-1 MODEL Setup

In [37]:
nixtla_client = NixtlaClient(api_key = 'nixtla-tok-IhSU5Nva61WLVkWHke9xSSHXCxTGrKiFB0f6LKcNied6jhkvdLIl0dEmBqdETcPDIk09rnQSaLmYcH8G')
historical_gpt_covar_cols = ['LAG_1_LOG_CASES', 'LAG_1_tmin_roll_2', 'LAG_1_prec_roll_2', 'end_of_month', 'LOG_CASES', 'MONTH']
time_gpt_covar_cols = ['LAG_1_LOG_CASES', 'LAG_1_tmin_roll_2', 'LAG_1_prec_roll_2', 'end_of_month', 'MONTH']

## TimeGPT + Deep TCN Historical forecasting

In [44]:
#Historical forecasting
for i in range(14): #Change 14 to be len(province_names)
    print(f"Processing province {i+1} out of 14")
    subset_df = timeseries_df[timeseries_df['PROVINCE'] == province_names[i]]
    subset_df = subset_df[:start_idx]
    #DEEP TCN
    target_series = [TimeSeries.from_dataframe(subset_df, 
                                           time_col = 'end_of_month',
                                           value_cols = case_cols)]

    covariate_series = [TimeSeries.from_dataframe(subset_df, 
                                              time_col = 'end_of_month',
                                              value_cols = covariate_cols)]
    covariates_scaled = scaler_covars.fit_transform(covariate_series)

    tcn_model.fit(series=target_series[0], 
                      past_covariates=covariates_scaled,
                     epochs=150,
                     verbose = False)
    tcn_backtest = tcn_model.historical_forecasts(
        series=target_series[0], 
        past_covariates=covariates_scaled,
        forecast_horizon=1,
        num_samples=2500,
        retrain=True,
        verbose=True)
    tmp = TimeSeries.all_values(tcn_backtest)
    tmp =tmp.reshape(43, 2500) #Need to change this 43 to not be hard-coded
    tcn_df = pd.DataFrame(tmp)
    
    tcn_df.to_csv(f'C:/Users/mills/Documents/peru_dengue/province/python/output/historical_tcn_dataframe_{i}.csv')

    
    #TIME GPT
    subset_df_copy = subset_df.copy()
    subset_df_copy = subset_df_copy[historical_gpt_covar_cols]
    time_gpt_covar_df = subset_df[time_gpt_covar_cols]
    historical_timegpt_fcst_df = nixtla_client.forecast(df=subset_df_copy, 
                                         X_df=time_gpt_covar_df.iloc[[-1]],
                                             finetune_steps = 10,
                                         h=1, quantiles=quantiles, time_col='end_of_month',
                                         target_col='LOG_CASES', add_history=True,)
    historical_timegpt_fcst_df.to_csv(f'C:/Users/mills/Documents/peru_dengue/province/python/output/historical_timegpt_dataframe_{i}.csv')
    
    # tcn_model.fit(series=target_series[0], 
    #                   past_covariates=covariates_scaled,
    #                  epochs=150,
    #                  verbose = False)
    # tcn_backtest = tcn_model.historical_forecasts(
    #     series=target_series[0], 
    #     past_covariates=covariates_scaled,
    #     forecast_horizon=1,
    #     num_samples = 2500,
    #     retrain=False,
    #     verbose=True)


INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Inferred freq: M
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Using future exogenous features: ['LAG_1_LOG_CASES', 'LAG_1_tmin_roll_2', 'LAG_1_prec_roll_2', 'MONTH']
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...


Processing province 1 out of 14


INFO:nixtla.nixtla_client:Calling Historical Forecast Endpoint...
INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Inferred freq: M
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Using future exogenous features: ['LAG_1_LOG_CASES', 'LAG_1_tmin_roll_2', 'LAG_1_prec_roll_2', 'MONTH']
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...


Processing province 2 out of 14


INFO:nixtla.nixtla_client:Calling Historical Forecast Endpoint...
INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Inferred freq: M
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Using future exogenous features: ['LAG_1_LOG_CASES', 'LAG_1_tmin_roll_2', 'LAG_1_prec_roll_2', 'MONTH']
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...


...

INFO:nixtla.nixtla_client:Calling Historical Forecast Endpoint...
INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Inferred freq: M
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Using future exogenous features: ['LAG_1_LOG_CASES', 'LAG_1_tmin_roll_2', 'LAG_1_prec_roll_2', 'MONTH']
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...


Processing province 13 out of 14


INFO:nixtla.nixtla_client:Calling Historical Forecast Endpoint...
INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Inferred freq: M
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Using future exogenous features: ['LAG_1_LOG_CASES', 'LAG_1_tmin_roll_2', 'LAG_1_prec_roll_2', 'MONTH']
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...


Processing province 14 out of 14


INFO:nixtla.nixtla_client:Calling Historical Forecast Endpoint...


## Historical SARIMA

In [27]:
#Historical SARIMA forecasting
for i in range(14):
    print(f"Processing province {i+1} out of 14")
    subset_df = timeseries_df[timeseries_df['PROVINCE'] == province_names[i]]
    subset_df = subset_df[:start_idx]

    target_series = [TimeSeries.from_dataframe(subset_df, 
                                           time_col = 'end_of_month',
                                           value_cols = case_cols)]
    series=target_series[0][:start_idx]
    
    future_cov = datetime_attribute_timeseries(series, "month", cyclic=True, add_length=6)
    model = StatsForecastAutoARIMA(season_length=12)
    model.fit(series)
    
    # Forecast the next 1 month (on scaled covariates)
    sarima_backtest = model.historical_forecasts(
        series=series, 
        forecast_horizon=1,
        num_samples=2500,
        verbose=True)
    tmp = TimeSeries.all_values(sarima_backtest)
    tmp =tmp.reshape(82, 2500) #Remove hard-coded 82 = number of predictions of backtest, 2500 = number of samples 
    sarima_df = pd.DataFrame(tmp)    
    sarima_df.to_csv(f'C:/Users/mills/Documents/peru_dengue/province/python/output/historical_sarima_dataframe_{i}.csv')

Processing province 1 out of 14


  0%|          | 0/82 [00:00<?, ?it/s]

Processing province 2 out of 14


  0%|          | 0/82 [00:00<?, ?it/s]

Processing province 3 out of 14


...

  0%|          | 0/82 [00:00<?, ?it/s]

Processing province 13 out of 14


  0%|          | 0/82 [00:00<?, ?it/s]

Processing province 14 out of 14


  0%|          | 0/82 [00:00<?, ?it/s]

## Testing Window SARIMA

In [23]:
counter = 0
for i in range(14):
    print(f"Processing province {i+1} out of 14")
    subset_df = timeseries_df[timeseries_df['PROVINCE'] == province_names[i]]
    dates = subset_df['end_of_month'].unique()
    target_series = [TimeSeries.from_dataframe(subset_df, 
                                           time_col = 'end_of_month',
                                           value_cols = case_cols)]
    start_idx = subset_df.shape[0] - 48  # Starting point for the 48-month horizon
    arima_predictions = []
    for j in range(48):
        print(f"Counter: {counter}")

        print(f"Processing month {j+1} out of 48")
        series=target_series[0][:start_idx + j]
        future_cov = datetime_attribute_timeseries(series, "month", cyclic=True, add_length=6)
        model = StatsForecastAutoARIMA(season_length=12)
        model.fit(series, future_covariates=future_cov)
        
        # Forecast the next 1 month (on scaled covariates)
        pred = model.predict(n=1, future_covariates=future_cov,
                                       num_samples = 2500)


        
        tmp = TimeSeries.all_values(pred[0])
        tmp =tmp.reshape(2500, 1)
        df = pd.DataFrame(tmp)
        arima_predictions.insert(j, df)
        counter = counter + 1
    arima_df = arima_predictions[0]
    for k in range(1, len(arima_predictions), 1):
        df = arima_predictions[k]
        arima_df = pd.concat([arima_df, df])
    arima_df.to_csv(f'C:/Users/mills/Documents/peru_dengue/province/python/output/arima_dataframe_{i}.csv')

Processing province 1 out of 14
Counter: 0
Processing month 1 out of 48
Counter: 1
Processing month 2 out of 48
Counter: 2
Processing month 3 out of 48
Counter: 3
Processing month 4 out of 48
Counter: 4
Processing month 5 out of 48
Counter: 5
Processing month 6 out of 48
Counter: 6
Processing month 7 out of 48
Counter: 7
Processing month 8 out of 48
Counter: 8
Processing month 9 out of 48
Counter: 9
Processing month 10 out of 48
Counter: 10
Processing month 11 out of 48
Counter: 11
Processing month 12 out of 48
Counter: 12
Processing month 13 out of 48
Counter: 13
Processing month 14 out of 48
Counter: 14
Processing month 15 out of 48
Counter: 15
Processing month 16 out of 48
Counter: 16
Processing month 17 out of 48
Counter: 17
Processing month 18 out of 48
Counter: 18
Processing month 19 out of 48
Counter: 19
Processing month 20 out of 48
Counter: 20
Processing month 21 out of 48
Counter: 21
Processing month 22 out of 48
Counter: 22
Processing month 23 out of 48
Counter: 23
Processin

## Testing Window TCN + TimeGPT

In [101]:
for i in range(14):
    print(f"Processing province {i+1} out of 14")
    subset_df = timeseries_df[timeseries_df['PROVINCE'] == province_names[i]]
    dates = subset_df['end_of_month'].unique()
    target_series = [TimeSeries.from_dataframe(subset_df, 
                                           time_col = 'end_of_month',
                                           value_cols = case_cols)]
    start_idx = subset_df.shape[0] - 48  # Starting point for the 48-month horizon

    covariate_series = [TimeSeries.from_dataframe(subset_df, 
                                              time_col = 'end_of_month',
                                              value_cols = covariate_cols)]
    
    tcn_predictions = []
    timegpt_predictions = []
    for j in range(48):
        print(f"Counter: {counter}")
        print(f"Processing month {j+1} out of 48")
        tmp = TimeSeries.all_values(target_series[0][:start_idx + j+1])
        true_cases.insert(counter, tmp[-1])
        subset_df_copy = subset_df.copy()
        subset_df_copy = subset_df_copy[:start_idx + j]
        subset_df_copy = subset_df_copy.drop('PROVINCE', axis = 1)        
        subset_df_copy = subset_df_copy[historical_gpt_covar_cols]
        time_gpt_covar_df = subset_df[time_gpt_covar_cols]
        time_gpt_covar_df = time_gpt_covar_df[:start_idx + j +1]
        timegpt_fcst_df = nixtla_client.forecast(df=subset_df_copy, 
                                         X_df=time_gpt_covar_df.iloc[[-1]], finetune_steps=10,
                                         h=1, quantiles=quantiles, time_col='end_of_month',
                                         target_col='LOG_CASES')
        timegpt_predictions.insert(j, timegpt_fcst_df)

        covariates_scaled = scaler_covars.fit_transform(covariate_series[:start_idx + j + 1])
        
        
        
        
        
        tcn_model_name = "TCN_Model"
        tcn_model = TCNModel(input_chunk_length=48,  # Use 48 months of history to predict the next month
                 output_chunk_length=1,  # Forecast 1 step ahead
                 kernel_size=3, 
                 num_filters=4, 
                 random_state=42, 
                   model_name=tcn_model_name,
                  likelihood = QuantileRegression(),
                save_checkpoints=True,
                     force_reset = True,
                             optimizer_kwargs={"lr": 1e-2},
                 n_epochs=150)
        tcn_model.fit(series=target_series[0][:start_idx + j], 
                      past_covariates=covariates_scaled[:start_idx + j],
                     epochs=150,
                     verbose = False)
        

        # Forecast the next 1 month (on scaled covariates)
        
        tcn_forecast_scaled = tcn_model.predict(n=1, past_covariates=covariates_scaled,
                                       num_samples = 2500)
        tmp = TimeSeries.all_values(tcn_forecast_scaled[0])
        tmp =tmp.reshape(2500, 1)
        df = pd.DataFrame(tmp)
        tcn_predictions.insert(j, df)

        
        counter = counter + 1 #Made forecasts for that individual time-province point
    
    
    #Save forecasts to df, then to csv
    
    #1) TCN
    tcn_df = tcn_predictions[0] #Initialise TCN predictions
    for k in range(1, len(tcn_predictions), 1):
        df = tcn_predictions[k]
        tcn_df = pd.concat([tcn_df, df])
    tcn_df.to_csv(f'C:/Users/mills/Documents/peru_dengue/province/python/output/tcn_dataframe_{i}.csv')

    #2) TimeGPT
    timegpt_df = timegpt_predictions[0]
    for k in range(1, len(timegpt_predictions), 1):
        df = timegpt_predictions[k]
        timegpt_df = pd.concat([timegpt_df, df])
    timegpt_df.to_csv(f'C:/Users/mills/Documents/peru_dengue/province/python/output/finetuned_timegpt_dataframe_{i}.csv')


INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Inferred freq: M
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Using future exogenous features: ['LAG_1_LOG_CASES', 'LAG_1_tmin_roll_2', 'LAG_1_prec_roll_2']
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...


Processing province 2 out of 14
Counter: 728
Processing month 1 out of 48


INFO:darts.models.forecasting.torch_forecasting_model:Train dataset contains 44 samples.
INFO:darts.models.forecasting.tcn_model:Number of layers chosen: 4
INFO:darts.models.forecasting.torch_forecasting_model:Time series values are 64-bits; casting model to float64.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer.fit` stopped: `max_epochs=150` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

INFO:darts.models.forecasting.torch_forecasting_model:Train dataset contains 44 samples.
INFO:darts.models.forecasting.torch_forecasting_model:Time series values are 64-bits; casting model to float64.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer.fit` stopped: `max_epochs=150` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


...

Counter: 775
Processing month 48 out of 48


INFO:darts.models.forecasting.torch_forecasting_model:Train dataset contains 91 samples.
INFO:darts.models.forecasting.tcn_model:Number of layers chosen: 4
INFO:darts.models.forecasting.torch_forecasting_model:Time series values are 64-bits; casting model to float64.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer.fit` stopped: `max_epochs=150` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

INFO:darts.models.forecasting.torch_forecasting_model:Train dataset contains 91 samples.
INFO:darts.models.forecasting.torch_forecasting_model:Time series values are 64-bits; casting model to float64.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer.fit` stopped: `max_epochs=150` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

## TimeGPT Without Covariates

In [40]:
#Historical Forecasting
no_covar_cols = ['end_of_month', 'LOG_CASES']
counter = 0
for i in range(14):
    print(f"Processing province {i+1} out of 14")
    subset_df = timeseries_df[timeseries_df['PROVINCE'] == province_names[i]]
    dates = subset_df['end_of_month'].unique()
    target_series = [TimeSeries.from_dataframe(subset_df, 
                                           time_col = 'end_of_month',
                                           value_cols = case_cols)]
    start_idx = subset_df.shape[0] - 48  # Starting point for the 48-month horizon
    subset_df_copy = subset_df.copy()
    subset_df_copy = subset_df_copy[no_covar_cols]
    historical_timegpt_fcst_df = nixtla_client.forecast(df=subset_df_copy, 
                                                        finetune_steps = 10,
                                                        h=1, quantiles=quantiles, time_col='end_of_month',
                                                        target_col='LOG_CASES', add_history=True)
    historical_timegpt_fcst_df.to_csv(f'C:/Users/mills/Documents/peru_dengue/province/python/output/historical_no_covars_timegpt_dataframe_{i}.csv')


INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Inferred freq: M
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...


Processing province 1 out of 14


INFO:nixtla.nixtla_client:Calling Historical Forecast Endpoint...
INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Inferred freq: M
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...


Processing province 2 out of 14


INFO:nixtla.nixtla_client:Calling Historical Forecast Endpoint...
INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Inferred freq: M
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...


...

INFO:nixtla.nixtla_client:Calling Historical Forecast Endpoint...
INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Inferred freq: M
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...


Processing province 13 out of 14


INFO:nixtla.nixtla_client:Calling Historical Forecast Endpoint...
INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Inferred freq: M
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...


Processing province 14 out of 14


INFO:nixtla.nixtla_client:Calling Historical Forecast Endpoint...


In [11]:
#Testing Window - TimeGPT Without covars
no_covar_cols = ['end_of_month', 'LOG_CASES']
counter = 0
for i in range(14):
    print(f"Processing province {i+1} out of 14")
    subset_df = timeseries_df[timeseries_df['PROVINCE'] == province_names[i]]
    dates = subset_df['end_of_month'].unique()
    target_series = [TimeSeries.from_dataframe(subset_df, 
                                           time_col = 'end_of_month',
                                           value_cols = case_cols)]
    start_idx = subset_df.shape[0] - 48  # Starting point for the 48-month horizon
    

    no_covar_timegpt_predictions = []
    for j in range(48):
        print(f"Counter: {counter}")

        print(f"Processing month {j+1} out of 48")
        subset_df_copy = subset_df.copy()
        subset_df_copy = subset_df_copy[:start_idx + j]
        subset_df_copy = subset_df_copy.drop('PROVINCE', axis = 1)        
        subset_df_copy = subset_df_copy[no_covar_cols]
        timegpt_fcst_df = nixtla_client.forecast(df=subset_df_copy, finetune_steps=10,
                                         h=1, quantiles=quantiles, time_col='end_of_month',
                                         target_col='LOG_CASES')
        # timegpt_predictions.insert(counter, timegpt_fcst_df)
        no_covar_timegpt_predictions.insert(j, timegpt_fcst_df)

        
    timegpt_df = no_covar_timegpt_predictions[0]
    for k in range(1, len(no_covar_timegpt_predictions), 1):
        df = no_covar_timegpt_predictions[k]
        timegpt_df = pd.concat([timegpt_df, df])
    timegpt_df.to_csv(f'C:/Users/mills/Documents/peru_dengue/province/python/output/finetuned_no_covars_timegpt_dataframe_{i}.csv')

INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Inferred freq: M
INFO:nixtla.nixtla_client:Querying model metadata...


Processing province 1 out of 14
Counter: 0
Processing month 1 out of 48


INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...
INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Inferred freq: M
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...


Counter: 0
Processing month 2 out of 48


INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Inferred freq: M
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...


...

Counter: 0
Processing month 46 out of 48


INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Inferred freq: M
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...


Counter: 0
Processing month 47 out of 48


INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Inferred freq: M
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...


Counter: 0
Processing month 48 out of 48
